In [1]:
import pandas as pd
import numpy as np
import os
import datetime
from haversine import haversine
import json


os.getcwd()

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

In [2]:
excel_file=pd.ExcelFile("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/zips_list_for_Xiao_test/BL SOTF Invest Level Test Brief Tables 20190510.xlsx")
excel_file.sheet_names


['Test Design',
 '2019 SOTF Lists by Wave',
 'Flight Summary',
 '2018 SOTF Openings',
 'Store List March 2019']

In [3]:
store_list_SFTP=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190501-135143-036.txt",
                              sep="|",dtype=str,usecols=['location_id','zip_cd','state_nm'])
store_list_SFTP=store_list_SFTP.drop_duplicates()
store_list_SFTP['zip_cd']=store_list_SFTP['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))
store_list_SFTP=store_list_SFTP.rename(columns={"state_nm":"ST"})
store_list_SFTP.head(2)


,location_id,ST,zip_cd
0,3,GA,30906
1,30,OH,43402


In [4]:
DMA_zips_nielsen=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",
                               dtype=str,skiprows=1,usecols=[0,2])
DMA_zips_nielsen=DMA_zips_nielsen.drop_duplicates().rename(columns={"CODE":"zip_cd","NAME":"Market"})
sorted(DMA_zips_nielsen['Market'].unique().tolist())
DMA_zips_nielsen=DMA_zips_nielsen[DMA_zips_nielsen['Market'].isin(['PHOENIX (PRESCOTT)','AUSTIN'])]
stores_Phe_Aus=pd.merge(store_list_SFTP,DMA_zips_nielsen,on="zip_cd",how="left")
stores_Phe_Aus=stores_Phe_Aus[~pd.isnull(stores_Phe_Aus['Market'])]

In [5]:
long_island_stores=excel_file.parse("Store List March 2019",dtype=str)
long_island_stores['Added_DMA_JL']=long_island_stores['Added_DMA_JL'].fillna("nan")
long_island_stores=long_island_stores[long_island_stores['Added_DMA_JL'].str.contains("Long Island")]
long_island_stores=long_island_stores[['Store']].drop_duplicates().rename(columns={"Store":"location_id"})
long_island_stores.shape

(9, 1)

In [6]:
long_island_stores=pd.merge(long_island_stores,store_list_SFTP[['location_id','zip_cd','ST']],on="location_id",how="left")
long_island_stores['Market']="Long Island"

In [7]:
output_1_DMAs=stores_Phe_Aus.append(long_island_stores)

SOTF_labels=excel_file.parse("Store List March 2019",dtype=str,usecols=['Store','SOTF']).rename(columns={"Store":"location_id"})
SOTF_labels=SOTF_labels[pd.notnull(SOTF_labels['location_id'])]
output_1_DMAs=pd.merge(output_1_DMAs,SOTF_labels,on="location_id",how="left")
output_1_DMAs=output_1_DMAs.rename(columns={"zip_cd":"store_zip"})

In [8]:
output_2_July_GO=excel_file.parse("2019 SOTF Lists by Wave",dtype=str,usecols=[26,27,28],skiprows=1)
output_2_July_GO.columns=['location_id','Market','ST']
output_2_July_GO['SOTF']="G.O. 0726"
output_2_July_GO=pd.merge(output_2_July_GO,store_list_SFTP[['location_id','zip_cd']],on="location_id",how="left").rename(columns={"zip_cd":"store_zip"})

In [9]:
missing_store_zips={}
missing_store_zips.update({"4699":"91803"})
missing_store_zips.update({"4679":"48864"})
missing_store_zips.update({"5392":"07652"})
missing_store_zips.update({"4668":"26651"})

output_2_July_GO_0=output_2_July_GO[pd.notnull(output_2_July_GO['store_zip'])]
output_2_July_GO_1=output_2_July_GO[pd.isnull(output_2_July_GO['store_zip'])]
output_2_July_GO_1['store_zip']=output_2_July_GO_1['location_id'].apply(lambda x: missing_store_zips[x])

output_2_July_GO=output_2_July_GO_0.append(output_2_July_GO_1)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [10]:
df_all_interested_stores=output_1_DMAs.append(output_2_July_GO)

In [11]:
store_lat_lng=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190501-135143-036.txt",
                              sep="|",dtype=str,usecols=['location_id','latitude_meas','longitude_meas'])
store_lat_lng['latitude_meas']=store_lat_lng['latitude_meas'].astype(float)
store_lat_lng['longitude_meas']=store_lat_lng['longitude_meas'].astype(float)

df_all_interested_stores=pd.merge(df_all_interested_stores,store_lat_lng,on="location_id",how="left")
df_google_latlng=df_all_interested_stores[pd.isnull(df_all_interested_stores['latitude_meas'])]
df_google_latlng

,Market,SOTF,ST,location_id,store_zip,longitude_meas,latitude_meas
114,"ALHAMBRA, CA",G.O. 0726,CA,4699,91803,NaN,NaN
115,"OKEMOS, MI",G.O. 0726,MI,4679,48864,NaN,NaN
116,"PARAMUS, NJ",G.O. 0726,NJ,5392,07652,NaN,NaN
117,"SUMMERSVILLE, WV",G.O. 0726,WV,4668,26651,NaN,NaN


In [12]:
new_store_add_dict={}
# 4699 		2500 WEST COMMONWEALTH AVENUE	FREMONT PLAZA	ALHAMBRA	CA	91803
new_store_add_dict.update({"4699":{"google_address":"2500 W Commonwealth Ave, Alhambra, CA 91803",
                                   "google_lat_lng":[34.0860813,-118.151011]}})

# 4679 		2020 W. GRAND RIVER AVE	BEST BUY CENTER	OKEMOS	MI	48864-1776
new_store_add_dict.update({"4679":{"google_address":"2020 W Grand River Rd, Okemos, MI 48864",
                                   "google_lat_lng":[42.7234646,-84.4235839]}})

# 5392 		634 N. STATE RT. 17	TOYS R US CENTER	PARAMUS	NJ	07652-3114
new_store_add_dict.update({"5392":{"google_address":"634 E Ridgewood Ave, Paramus, NJ 07652",
                                   "google_lat_lng":[40.9638559,-74.0761499]}})

# 4668 		197 MECHANTS WALK	MERCHANTS WALK SHOPPING CENTER	SUMMERSVILLE	WV	26651
new_store_add_dict.update({"4668":{"google_address":"197 Merchant Walk, Summersville, WV 26651",
                                   "google_lat_lng":[38.2983025,-80.8343841]}})


df_all_interested_stores_0=df_all_interested_stores[pd.notnull(df_all_interested_stores['latitude_meas'])]
df_all_interested_stores_1=df_all_interested_stores[pd.isnull(df_all_interested_stores['latitude_meas'])]
df_all_interested_stores_1['latitude_meas']=df_all_interested_stores_1['location_id'].apply(lambda x: new_store_add_dict[x]['google_lat_lng'][0])
df_all_interested_stores_1['longitude_meas']=df_all_interested_stores_1['location_id'].apply(lambda x: new_store_add_dict[x]['google_lat_lng'][1])

df_all_interested_stores=df_all_interested_stores_0.append(df_all_interested_stores_1)
df_all_interested_stores.shape

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(118, 7)

In [13]:
store_level_zips=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/sales_by_zip (Store level).csv",
                             dtype=str,usecols=['location_id','revenue_flag','zip'])
store_level_zips=store_level_zips[store_level_zips['revenue_flag'].isin(['P','S'])]
df_all_interested_stores=pd.merge(df_all_interested_stores,store_level_zips,on="location_id",how="left")

df_all_interested_stores_old=df_all_interested_stores[pd.notnull(df_all_interested_stores['revenue_flag'])]
df_all_interested_stores_new=df_all_interested_stores[pd.isnull(df_all_interested_stores['revenue_flag'])]


In [14]:
del df_all_interested_stores_new['zip']
del df_all_interested_stores_new['revenue_flag']
df_all_interested_stores_new=df_all_interested_stores_new.reset_index()
del df_all_interested_stores_new['index']

df_all_new_zips=pd.DataFrame()
for ind,row in df_all_interested_stores_new.iterrows():
    location_id=row['location_id']
    store_center=(row['latitude_meas'],row['longitude_meas'])
    for zip_cd in zip_centers.keys():
        dist=haversine(store_center,zip_centers[zip_cd],miles=True)
        if dist<=10:
            df=pd.DataFrame({"location_id":location_id,"zip":zip_cd,"revenue_flag":"zip_in_10","dist_miles":dist},index=[location_id])
            df_all_new_zips=df_all_new_zips.append(df)
            
    

In [15]:
df_all_interested_stores_new=pd.merge(df_all_interested_stores_new,df_all_new_zips,on="location_id",how="left")

In [16]:
df_all_interested_stores_old=df_all_interested_stores_old.reset_index()
del df_all_interested_stores_old['index']
df_all_interested_stores_old['dist_miles']=np.nan
for ind,row in df_all_interested_stores_old.iterrows():
    zip_cd=row['zip']
    store_center=(row['latitude_meas'],row['longitude_meas'])
    try:
        dist=haversine(store_center,zip_centers[zip_cd],miles=True)
        df_all_interested_stores_old.loc[ind,'dist_miles']=dist
    except:
        df_all_interested_stores_old.loc[ind,'dist_miles']=np.nan
        
df_all_interested_stores_old=df_all_interested_stores_old[(df_all_interested_stores_old['revenue_flag']=="P") | (df_all_interested_stores_old['dist_miles']<20)]

In [17]:
df_all=df_all_interested_stores_old.append(df_all_interested_stores_new).sort_values(["Market",'SOTF','location_id'])
col_orders=['location_id','Market','ST','SOTF','store_zip','latitude_meas','longitude_meas','revenue_flag','zip','dist_miles']
df_all=df_all[col_orders]


df_output_1=df_all[df_all['SOTF']!="G.O. 0726"]
df_output_2=df_all[df_all['SOTF']=="G.O. 0726"]
print(df_output_1.shape)
print(df_output_2.shape)
df_all_wide=df_all.groupby(['location_id','Market','ST','SOTF','store_zip','latitude_meas','longitude_meas'])['zip'].apply(set).to_frame().reset_index()

os.getcwd()

(976, 10)
(1434, 10)


'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/zips_list_for_Xiao_test'

In [23]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/zips_list_for_Xiao_test/BL_zips_for_test_stores_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

df_output_1.to_excel(writer,"df_1_3_Mkts",index=False)
df_output_2.to_excel(writer,"df_2_GO_SOTF_0726",index=False)

df_all_wide=df_all.groupby(['location_id','Market','ST','SOTF','store_zip','latitude_meas','longitude_meas'])['zip'].apply(set).to_frame().reset_index()
df_all_wide.to_excel(writer,"wide_by_store",index=False)

writer.save()